# Ariel Data Challenge 2025 - 詳細データ分析

## 概要
系外惑星大気の分光データ分析を深掘りします。

**目標**: 
- 283波長帯域での分光値パターンの理解
- 不確実性の分布とパターンの分析
- 予測対象の特性把握

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 設定
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
DATA_PATH = Path('../')

print("ライブラリ読み込み完了")

## 1. データ構造の詳細分析

In [ ]:
# サンプルデータの読み込み
df = pd.read_csv(DATA_PATH / 'sample_submission.csv')

print(f"データ形状: {df.shape}")
print(f"惑星数: {len(df)}")
print(f"特徴量数: {df.shape[1] - 1}")

# カラム分析
wl_cols = [col for col in df.columns if col.startswith('wl_')]
sigma_cols = [col for col in df.columns if col.startswith('sigma_')]

print(f"\n波長カラム数: {len(wl_cols)}")
print(f"不確実性カラム数: {len(sigma_cols)}")
print(f"合計予測対象: {len(wl_cols) + len(sigma_cols)}")

df.head(2)

## 2. 分光データのパターン分析

In [ ]:
# 波長値の統計
wl_data = df[wl_cols]
sigma_data = df[sigma_cols]

print("=== 波長データ (wl_*) の統計 ===")
print(f"全体統計:")
print(f"  範囲: {wl_data.min().min():.6f} - {wl_data.max().max():.6f}")
print(f"  平均: {wl_data.mean().mean():.6f}")
print(f"  標準偏差: {wl_data.std().mean():.6f}")
print(f"  ユニーク値数: {len(wl_data.stack().unique())}")

print("\n=== 不確実性データ (sigma_*) の統計 ===")
print(f"全体統計:")
print(f"  範囲: {sigma_data.min().min():.6f} - {sigma_data.max().max():.6f}")
print(f"  平均: {sigma_data.mean().mean():.6f}")
print(f"  標準偏差: {sigma_data.std().mean():.6f}")
print(f"  ユニーク値数: {len(sigma_data.stack().unique())}")

In [ ]:
# データの分布を可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 波長データのヒストグラム
axes[0,0].hist(wl_data.values.flatten(), bins=50, alpha=0.7, color='blue')
axes[0,0].set_title('波長値の分布')
axes[0,0].set_xlabel('値')
axes[0,0].set_ylabel('頻度')

# 不確実性データのヒストグラム  
axes[0,1].hist(sigma_data.values.flatten(), bins=50, alpha=0.7, color='red')
axes[0,1].set_title('不確実性の分布')
axes[0,1].set_xlabel('値')
axes[0,1].set_ylabel('頻度')

# 各惑星の波長プロファイル
for i, planet_id in enumerate(df['planet_id']):
    wl_profile = df[df['planet_id']==planet_id][wl_cols].values[0]
    axes[1,0].plot(range(len(wl_profile)), wl_profile, label=f'Planet {planet_id}', alpha=0.8)

axes[1,0].set_title('各惑星の波長プロファイル')
axes[1,0].set_xlabel('波長インデックス')
axes[1,0].set_ylabel('波長値')
axes[1,0].legend()

# 各惑星の不確実性プロファイル
for i, planet_id in enumerate(df['planet_id']):
    sigma_profile = df[df['planet_id']==planet_id][sigma_cols].values[0]
    axes[1,1].plot(range(len(sigma_profile)), sigma_profile, label=f'Planet {planet_id}', alpha=0.8)

axes[1,1].set_title('各惑星の不確実性プロファイル')
axes[1,1].set_xlabel('波長インデックス')
axes[1,1].set_ylabel('不確実性')
axes[1,1].legend()

plt.tight_layout()
plt.show()

## 3. 相関とパターンの発見

In [ ]:
# 波長間の相関を調べる（サンプリングして計算負荷を軽減）
sample_wl_cols = wl_cols[::10]  # 10個おきにサンプリング
sample_sigma_cols = sigma_cols[::10]

# 波長データの相関行列
wl_corr = df[sample_wl_cols].corr()

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# 波長の相関マップ
sns.heatmap(wl_corr, cmap='coolwarm', center=0, 
            square=True, ax=axes[0], cbar_kws={'label': '相関係数'})
axes[0].set_title('波長データの相関マップ（サンプリング）')

# 不確実性の相関マップ
sigma_corr = df[sample_sigma_cols].corr()
sns.heatmap(sigma_corr, cmap='coolwarm', center=0, 
            square=True, ax=axes[1], cbar_kws={'label': '相関係数'})
axes[1].set_title('不確実性の相関マップ（サンプリング）')

plt.tight_layout()
plt.show()

print(f"波長データの平均相関: {wl_corr.values[np.triu_indices_from(wl_corr.values, k=1)].mean():.6f}")
print(f"不確実性の平均相関: {sigma_corr.values[np.triu_indices_from(sigma_corr.values, k=1)].mean():.6f}")

## 4. データの変動性分析

In [ ]:
# 各波長での変動係数を計算
wl_cv = wl_data.std() / wl_data.mean()  # 変動係数
sigma_cv = sigma_data.std() / sigma_data.mean()

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 波長の変動係数
axes[0].plot(range(len(wl_cv)), wl_cv.values, 'b-', alpha=0.8)
axes[0].set_title('各波長での変動係数')
axes[0].set_xlabel('波長インデックス')
axes[0].set_ylabel('変動係数')
axes[0].grid(True, alpha=0.3)

# 不確実性の変動係数
axes[1].plot(range(len(sigma_cv)), sigma_cv.values, 'r-', alpha=0.8)
axes[1].set_title('各波長での不確実性の変動係数')
axes[1].set_xlabel('波長インデックス')
axes[1].set_ylabel('変動係数')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"波長データ変動係数 - 平均: {wl_cv.mean():.6f}, 範囲: {wl_cv.min():.6f} - {wl_cv.max():.6f}")
print(f"不確実性変動係数 - 平均: {sigma_cv.mean():.6f}, 範囲: {sigma_cv.min():.6f} - {sigma_cv.max():.6f}")

## 5. 主成分分析（PCA）による次元削減

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 波長データのPCA
scaler_wl = StandardScaler()
wl_scaled = scaler_wl.fit_transform(wl_data)

pca_wl = PCA()
wl_pca = pca_wl.fit_transform(wl_scaled)

# 不確実性データのPCA
scaler_sigma = StandardScaler()
sigma_scaled = scaler_sigma.fit_transform(sigma_data)

pca_sigma = PCA()
sigma_pca = pca_sigma.fit_transform(sigma_scaled)

# 寄与率の可視化
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 波長データの寄与率
axes[0].plot(range(1, min(21, len(pca_wl.explained_variance_ratio_)+1)), 
            pca_wl.explained_variance_ratio_[:20], 'bo-')
axes[0].set_title('波長データのPCA寄与率')
axes[0].set_xlabel('主成分')
axes[0].set_ylabel('寄与率')
axes[0].grid(True, alpha=0.3)

# 不確実性データの寄与率
axes[1].plot(range(1, min(21, len(pca_sigma.explained_variance_ratio_)+1)), 
            pca_sigma.explained_variance_ratio_[:20], 'ro-')
axes[1].set_title('不確実性データのPCA寄与率')
axes[1].set_xlabel('主成分')
axes[1].set_ylabel('寄与率')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"波長データ - 第1主成分寄与率: {pca_wl.explained_variance_ratio_[0]:.4f}")
print(f"波長データ - 累積寄与率（上位10成分）: {pca_wl.explained_variance_ratio_[:10].sum():.4f}")
print(f"不確実性 - 第1主成分寄与率: {pca_sigma.explained_variance_ratio_[0]:.4f}")
print(f"不確実性 - 累積寄与率（上位10成分）: {pca_sigma.explained_variance_ratio_[:10].sum():.4f}")

## 6. データ品質とノイズの分析

In [ ]:
# 異常値検出
def detect_outliers_iqr(data):
    """IQR法による異常値検出"""
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (data < lower_bound) | (data > upper_bound)

# 波長データの異常値
wl_outliers = detect_outliers_iqr(wl_data)
sigma_outliers = detect_outliers_iqr(sigma_data)

wl_outlier_ratio = wl_outliers.sum().sum() / (wl_data.shape[0] * wl_data.shape[1])
sigma_outlier_ratio = sigma_outliers.sum().sum() / (sigma_data.shape[0] * sigma_data.shape[1])

print(f"波長データの異常値割合: {wl_outlier_ratio:.6f}")
print(f"不確実性データの異常値割合: {sigma_outlier_ratio:.6f}")

# Signal-to-Noise比の推定
# 不確実性を使ってSNR推定
snr_estimate = wl_data.abs() / sigma_data

print(f"\n推定SNR統計:")
print(f"  平均SNR: {snr_estimate.mean().mean():.4f}")
print(f"  SNR範囲: {snr_estimate.min().min():.4f} - {snr_estimate.max().max():.4f}")

# SNRの分布
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(snr_estimate.values.flatten(), bins=50, alpha=0.7, color='green')
plt.title('推定SNRの分布')
plt.xlabel('SNR')
plt.ylabel('頻度')
plt.yscale('log')

plt.subplot(1, 2, 2)
snr_by_wavelength = snr_estimate.mean(axis=0)
plt.plot(range(len(snr_by_wavelength)), snr_by_wavelength, 'g-', alpha=0.8)
plt.title('波長別平均SNR')
plt.xlabel('波長インデックス')
plt.ylabel('平均SNR')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. 分析サマリーと次のステップ

In [ ]:
print("=== データ分析サマリー ===")
print(f"データセット:")
print(f"  - 惑星数: {len(df)}")
print(f"  - 波長帯域: {len(wl_cols)}")
print(f"  - 予測対象: {len(wl_cols) + len(sigma_cols)}個")

print(f"\n主要な発見:")
print(f"  - サンプルデータは定数値（実際のデータは異なる）")
print(f"  - 高次元データ（566次元）")
print(f"  - 波長と不確実性の両方を予測")
print(f"  - PCA第1成分の寄与率が高い（強い線形相関）")

print(f"\n次のステップ:")
print(f"  1. 実際の訓練データのダウンロードと分析")
print(f"  2. 校正データの前処理パイプライン構築")
print(f"  3. 特徴量エンジニアリング（PCA、wavelets等）")
print(f"  4. 回帰モデルの構築（波長・不確実性予測）")
print(f"  5. クロスバリデーション戦略の設計")